### Issue:  
Yelp gets their menu information from Locu (and potentially other sources), so they won't share that via their API.
http://techcrunch.com/2013/04/23/yelp-partners-with-locu-allowing-businesses-to-post-menus-daily-specials-photos-to-yelp-in-real-time/

In [11]:
import requests
import pandas as pd
import numpy as np 
import json 


def get_restaurants(location):
    
    key = 'iI4dsdXt-g13ESGVicCEP7g8svwE657yNKQtQx0UXtrjgteJuPjddz_RYf92YIWiUJxpMx_To53E2hXoMQmnpOSc3Jws0L7SAQTH2Qdno9XFlOih236mUlx6AIEhW3Yx'
    auth_header = {'Authorization': 'Bearer ' + key}
    url = 'https://api.yelp.com/v3/businesses/search'
    
    parameters = {"latitude":location["latitude"],"longitude":location["longitude"],"radius": 15,"limit":30,"sort_by":"rating","open_now":False}
    resp = requests.get(url, headers=auth_header, params=parameters)
    data = resp.json()
       
    df = pd.DataFrame(data['businesses'])
    df = pd.json_normalize(data['businesses'])
    df.drop('categories', axis='columns', inplace=True)
    df.drop('transactions', axis='columns', inplace=True)
    df.drop('image_url', axis='columns', inplace=True)
    df.drop('location.address2', axis='columns', inplace=True)
    df.drop('location.address3', axis='columns', inplace=True)
    df.drop('location.display_address', axis='columns', inplace=True)
    df.drop('phone', axis='columns', inplace=True)
    df = df.sort_values(['rating','review_count'], ascending=[False, False])
    df = df.head(4)
    names = list(np.array(df["name"].tolist()))
    rating = list(np.array(df["rating"].tolist()))
    numReviews = list(np.array(df["review_count"].tolist()))
    id_ = list(np.array(df["id"].tolist()))
    
    result=list()
    for i in range(4):
        info = {"rating":'',"review_count":0,'pop_reviews':[]}
        
        n = names[i]
        info["rating"] = rating[i]
        info["review_count"] = numReviews[i]
        
        rev_url = 'https://api.yelp.com/v3/businesses/{}/reviews'.format(id_[i])
        resp_ = requests.get(rev_url,headers=auth_header)
        datum = resp_.json()
        reviews = pd.DataFrame(datum["reviews"])   
        reviews = reviews.sort_values(['rating'], ascending=[False])

        if len(reviews)>4:
            reviews = reviews.head(4)
        else:
            pass
        reviews_ = reviews["text"].to_list()
        info["pop_reviews"] = "1. "+reviews_[0] +'; 2. '+ reviews_[1] +'; 3. '+ reviews_[2]
        info['name'] = n
        result.append(info)
 
    # key = restaurant name
    # values = {rating:int, review_count:int, popolur_reviews:string}
    df = pd.DataFrame.from_dict(result)
    df = df[['name', 'rating', 'review_count', 'pop_reviews']]
    return df

In [12]:
# test
location = {'latitude': 40.7303859, 'longitude': -73.9860613} #coordinate -> dictionary(or json blob)
get_restaurants(location)

,name,rating,review_count,pop_reviews
0,LoveMama,4.5,5138,1. Came back yesterday after 3 years!!! Sorry ...
1,Numero 28 Pizzeria Napoletana,4.0,533,1. This location has such a freeing vibe! \n\n...
2,Insomnia Cookies,3.5,174,1. I love their cookies. It's so good and deli...
3,Pangea,3.5,136,1. A great place. Great to come see MUSIC here...
